# Imoport necessary modules

In [ ]:
import numpy as np
import sep
import astropy.io 

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

# set the size of the image with rcParams
rcParams['figure.figsize'] = [10., 8.]

# Open the fits file and extract data

In [ ]:
hdul = fits.open("image.fits")

data = hdul[0].data

# Display image

In [ ]:
m,s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

# Save the image as a png file
plt.savefig('figone.png')

In [ ]:
# Set a variable name for the image background
bkg = sep.Background(data)


In [ ]:
# Display global background level and global background rms
print(bkg.globalback)
print(bkg.globalrms)



In [ ]:
# Create an array of the background
bkg_image = bkg.back()

# Display only the background from the image

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

# Save the image background as a png file
plt.savefig('figtwo.png')

In [ ]:
# Create an array of the background rms
bkg_rms = bkg.rms()

# Display only the background rms

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

# Save the background rms as a png file
plt.savefig('figthree.png')

# Separate the data from the image background

In [ ]:
data_sub = data - bkg


# Extract the objects from the image

In [ ]:
# Define source of objects, set threshold of object detection, set variance for detection threshold
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)


# List the number of detected objects from the array

In [ ]:
len(objects)


# Show the background-subtracted image and create shapes to highlight each detected object

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
# Save image as png file 
plt.savefig('figfour.png')

# Display the parameters extracted from the objects array

In [ ]:
objects.dtype.names

# Extract the location and flux data from each object

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)


# Create loop to display the flux from the first 10 detected objects

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))